# Intro to Prompt Engioneering

This is a replacement notebook to use Claude Haiku from Amazon Bedrock 

To refer to original notebook using GPT v3.5 Turbo from OpenAI - [Chapter 04 - Intro_to_Prompt_Engineering_Tips_and_Tricks.ipynb](../notebooks/Chapter%2004%20-%20Intro_to_Prompt_Engineering_Tips_and_Tricks.ipynb) 

> If you are running the notebook understand AWS resources (e.g. EC2, SageMaker Studio Notebooks, etc.), you don't need to setup AWS profile. Otherwise, you may use AWS credentials, please create AWS profile at your local and replace `<<profile name>>` with the proper name. 

In [1]:
%env AWS_PROFILE=bedrock_profile
%env AWS_DEFAULT_REGION=us-east-1

env: AWS_PROFILE=bedrock_profile
env: AWS_DEFAULT_REGION=us-east-1


Setting auto reload so as to ease the utility loading.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from scripts.bedrock_utils import completion
from langchain_core.prompts import ChatPromptTemplate


### Story Generation

In [4]:
prompt_system = "You are a helpful assistant whose goal is to help write stories."

prompt = """Continue the following story. Write no more than 50 words.

Once upon a time, in a world where animals could speak, a courageous \
mouse named Benjamin decided to"""

messages = [
    ("system", prompt_system),
    ("human", prompt)
]

prompt_template = ChatPromptTemplate.from_messages(messages)

response = completion(prompt_template, temperature=1)
print(response)

Once upon a time, in a world where animals could speak, a courageous mouse named Benjamin decided to embark on a grand adventure. He gathered his friends, a wise owl and a nimble squirrel, and together they set out to explore the enchanted forest, determined to uncover its hidden secrets.


### Product Description

In [5]:
system_prompt = """
You are a helpful assistant whose goal is to help write product descriptions.
"""

prompt = """
Write a captivating product description for a luxurious, handcrafted, 
limited-edition fountain pen made from rosewood and gold.
Write no more than 40 words.
"""

messages = [
    ("system", prompt_system),
    ("human", prompt)
]
prompt_template = ChatPromptTemplate.from_messages(messages)

response = completion(prompt_template)
print(response)

Elevate your writing experience with our exquisite rosewood and gold fountain pen. Crafted with meticulous attention to detail, this limited-edition masterpiece combines timeless elegance and exceptional performance, making it a true work of art.


### Zero-Shot Prompting

In [6]:
prompt_system = "You are a helpful assistant whose goal is to write short peoms."

prompt = "Write a short peom about {topic}."

messages = [
    ("system", prompt_system),
    ("human", prompt)
]

parameters = {"topic": "summer"}

prompt_template = ChatPromptTemplate.from_messages(messages)

response = completion(prompt_template, parameters)
print(response)


Here is a short poem about summer:

Warm summer days, bright and long,
Filled with laughter, joy, and song.
Lazy afternoons by the sea,
Waves lapping, a gentle symphony.

Fireflies dance in the evening air,
Crickets chirping without a care.
Juicy fruits and flowers in bloom,
Nature's vibrant, colorful tune.

Memories made, moments to treasure,
Summer's embrace, a timeless pleasure.
Embracing the sun's golden glow,
Savoring the season's ebb and flow.


### In-context learning

In [7]:
prompt = "Write a short poem about {topic}."

examples = {
    "nature": """Birdsong fills the air,\nMountains high and valleys deep,\nNature's music sweet.""",
    "winter": """Snow blankets the ground,\nSilence is the only sound,\nWinter's beaty found."""
}

messages = [
    ("system", prompt_system),
    ("human", prompt.format(topic="nature")),
    ("ai", examples['nature']),
    ("human", prompt.format(topic="winter")),
    ("ai", examples['winter']),
    ("human", prompt.format(topic="summer"))
]

prompt_template = ChatPromptTemplate.from_messages(messages)
response = completion(prompt_template)
print(response)

Warm sun on my face,
Waves lapping at the shore,
Summer's peaceful grace.


In [8]:
from langchain import PromptTemplate, FewShotPromptTemplate
examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""
example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""Here are some examples of colors and the emotions associated with \
them:\n\n""",
    suffix="""\n\nNow, given a new color, identify the emotion associated with \
it. Just color and emotion on output:\n\nColor: {input}\nEmotion:""",
    input_variables=["input"],
    example_separator="\n",
)

response = completion(few_shot_prompt, {"input": "purpose"})
print(response)


Color: purpose
Emotion: determination


### Chain Prompting

> [Notes] I don't quite like the way of chaining prompts - a previous output is an input for the next. Will setup a todo task to investigation better practices.

In [9]:
# prompt 1
template_question = """
What is the name of the famous scientist who developed the theory of general relativity?
Answer (include name only, nothing else):
"""
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# prompt 2
tempalte_fact = """
Provide a brief description of {scientist}'s theory of general relativity.
Answer:
"""
prompt_fact = PromptTemplate(template=tempalte_fact, input_variables=['scientist'])

scientist = completion(prompt_question)

completion(prompt_fact, {"scientist": scientist})

"Albert Einstein's theory of general relativity is a fundamental theory in physics that describes the nature of gravity and its relationship to the structure of space and time. The key ideas of the theory are:\n\n1. Gravity is not a force, but a consequence of the curvature of spacetime. The presence of mass and energy causes a distortion or curvature in the fabric of spacetime, and this curvature is what we experience as the force of gravity.\n\n2. The motion of objects is determined by the curvature of spacetime. Massive objects, like planets and stars, curve the spacetime around them, and other objects, like smaller planets or satellites, follow the curvature of this spacetime.\n\n3. The speed of light is constant in all reference frames. This is a fundamental principle of the theory, which has important implications for our understanding of space, time, and the nature of the universe.\n\n4. Space and time are not absolute, but are relative to the observer. The theory of general rel

### Bad Prompt Practices

In [10]:
template = "Tell me something about {topic}."
prompt = PromptTemplate(template=template, input_variables=["topic"])
prompt.format(topic="dog")

'Tell me something about dog.'

In [11]:
template_fact = """
Tell me something interesting about {scientist}
Answer:
"""
prompt_fact = PromptTemplate(template=template_fact, input_variables=["scientist"])
completion(prompt_fact, {"scientist": scientist})

'Here are some interesting facts about Albert Einstein:\n\n1. Einstein\'s famous equation E=mc^2 shows the relationship between energy (E), mass (m), and the speed of light (c). This equation demonstrates that energy and mass are interchangeable.\n\n2. Einstein received the Nobel Prize in Physics in 1921, not for his theory of relativity, but for his discovery of the law of the photoelectric effect, which was an important foundation for the development of quantum theory.\n\n3. Einstein was born in Germany but later renounced his German citizenship. He was a pacifist and a humanitarian who spoke out against nationalism and racism. He refused to work on the atomic bomb during World War II.\n\n4. Einstein had a very distinctive hairstyle - wild, unkempt curly hair. This became part of his iconic image as the brilliant, eccentric scientist. He often joked that he didn\'t have time to get his hair cut.\n\n5. Despite his brilliance, Einstein struggled in school as a child. He was a late talk

### Tips for Effective Prompting

With few-shot prompting, `Claude Haiku` (the default FM model we are using) seems to be keen to provide more informative response in the below example. I've provided two editions on prompt setting at suffix, with which you may need to steer Claude to align to examples on specific areas. In addition, I shared a 'chat' style prompting at the end.

* Example 1: Simple suffix leads to informative response.

In [12]:
examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    },
    {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    template=example_template, input_variables=["query", "answer"]
)

prefix = """
The following are excerpts from conversations with an AI life coach. The assistant provides insightful and
practical advice to the users' questions. Here are some examples:
"""

suffix = """
User: {query}
AI:
"""

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)
# few_shot_prompt_template.format_prompt()

parameters = {"query": "What are some tips for improving communication skills?"}

response = completion(few_shot_prompt_template, parameters)
print(response)

Here are some tips for improving communication skills based on the AI life coach's advice:

User: What are some tips for improving communication skills?
AI: Here are some helpful tips for improving your communication skills:

- Practice active listening. Make eye contact, ask clarifying questions, and paraphrase what the other person is saying to show you are engaged and understanding them.

- Work on your body language. Stand or sit with an open, confident posture. Use hand gestures and facial expressions to emphasize your points.

- Speak clearly and concisely. Avoid filler words like "um" and "uh." Get to the point and organize your thoughts before speaking.

- Learn to read social cues. Pay attention to the other person's tone, facial expressions, and body language to understand how they are responding.

- Be empathetic and try to see things from the other person's perspective. This can help you communicate in a more understanding and relatable way.

- Practice, practice, practice.

* Example 2: Be specific on the expected output aginst the context (/examples)

In [13]:
suffix = """
Please answer the below question as concise as the above examples:
User: {query}
AI:
"""

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)
# few_shot_prompt_template.format_prompt()

parameters = {"query": "What are some tips for improving communication skills?"}

response = completion(few_shot_prompt_template, parameters)
print(response)

Here is a concise response to the question about improving communication skills:

User: What are some tips for improving communication skills?
AI: Practice active listening, speak clearly and confidently, and work on empathy and understanding others' perspectives.


* Example 3: 'Chat' style prompting.

In [14]:
system_prompt = prefix

messages = [
    ("system", system_prompt),
    ("human", examples[0]['query']),
    ("ai", examples[0]['answer']),
    ("human", examples[1]['query']),
    ("ai", examples[1]['answer']),
    ("human", "What are some tips for improving communication skills?")
]

prompt_template = ChatPromptTemplate.from_messages(messages)
response = completion(prompt_template)
print(response)

Some tips for improving communication skills include active listening, asking thoughtful questions, and being aware of your body language and tone.
